In [1]:
%matplotlib inline

import sys 
import os 
import numpy as np

nb_dir = os.getcwd()
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from plotlib.loaders import *
from plotlib.plotters import *

from phdconf import config 

ylims=[0.67, 0.45, 0.60, 0.59, 0.38]

In [2]:
queries = load_queries(config.AUS_TOPIC_PATH)
tt_folds = read_folds('ausnl-folds.txt')

In [3]:
BASE_DIR = os.path.join(os.environ["HOME"], 'phd-generated')

In [4]:
index_names = ['case-topics']
qrel_paths = [config.AUS_QREL_PATH] * len(index_names)
rel_levels = [config.AUS_REL_LEVEL] * len(index_names)
display_names = ['base', 'sec-count', 'leg-count', 'date', 'hierarchy']

topics = 'case-topics'

metrics = copy.copy(config.METRIC_NAMES)
del metrics['recall_100']

to = 99

In [5]:
mu = 1050
base_df = load_1d_dfs(['filtered-phrasestop'], qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, mu, mu, 1)[0][0]
sdm_base_df = load_1d_dfs(['filtered-phrasestop'], qrel_paths, os.path.join(BASE_DIR, 'sdm_rerank'), 'case-topics-{0}-sdm_rerank-dir-mu-{1:.2f}-weights-0.85-0.00-0.15-window-2.run', rel_levels, mu, mu, 1)[0][0]
base_qry = load_1d_dfs(['filtered-phrasestop'], qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, mu, mu, 1, per_query=True)[0][0]

In [6]:
# class Linker:
#     def __init__(self, path: str, id_path: str):
#         self._lookup = {}
#         with open(id_path) as f:
#             for line in f:
#                 parts = line.split()
#                 self._lookup[parts[0].upper()] = int(parts[1])

#         self._lookup_ind = ['']*len(self._lookup)
#         for k, v in self._lookup.items():
#             self._lookup_ind[v] = k
        
#         self._inlinks = [None]*len(self._lookup)
#         self._outlinks = [None]*len(self._lookup)

#         with open(path) as f:
#             for line in f:
#                 parts = list(map(int, line.split()))
                
#                 if self._inlinks[parts[1]] == None:
#                     self._inlinks[parts[1]] = []
#                 self._inlinks[parts[1]].append((parts[0], parts[2]))

#                 if self._outlinks[parts[0]] == None:
#                     self._outlinks[parts[0]] = []
#                 self._outlinks[parts[0]].append((parts[1], parts[2]))
                
#     def ids(self):
#         return self._lookup_ind
    
#     def out_links(self): 
#         return self._outlinks
    
#     def in_links(self): 
#         return self._inlinks
                
#     def get_links_for_id(_id: str, vals, count:bool = False):
#         ind = self._lookup.get(_id.toupper(), None)
#         if ind != None:
#             if count:
#                 _sum = 0
#                 for i in vals[ind]:
#                     _sum += i[1]
#                 return _sum
#             return len(vals[ind])
#         return 0

# linker = Linker(os.path.join(os.environ["HOME"], "go/src/cit-extract/links.txt"), os.path.join(os.environ["HOME"], "go/src/cit-extract/id-lookup.txt"))

In [7]:
# # get list of documents and relevance list 
# def get_qrel_rel_docs(path: str): 
#     out = set()
#     with open(path) as f:
#         for line in f:
#             parts = line.strip().split()
#             if int(parts[3]) > 0:
#                 out.add(parts[2])
                
#     return out

# def get_bin(v, bins): 
#     for i in range(1, len(bins)): 
# #         print(v, bins)
#         if v < bins[i]: 
#             return i-1
#     return 0 

# def get_rel_vals(qrels, lookup, bins):
#     vals = [0.0] * len(qrels)
#     for i, q in enumerate(qrels):
#         vals[i] = lookup.get(q, True)
    
#     bin_cnt = [0] * (len(bins)-1)
#     for i, v in enumerate(vals): 
#         bin_cnt[get_bin(v, bins)] +=1
    
#     return bin_cnt
    
# def get_buckets(data):
#     print(data[:10])
#     df = pd.DataFrame(data)
#     bucketed, bins = pd.cut(df[0], 100, retbins=True)
#     return bucketed, bins

In [8]:
# buckets = 54

# rel_docs = get_qrel_rel_docs(config.AUS_QREL_PATH)

# in_links = linker.in_links()
# out_links = linker.out_links()

# inlink = [len(x) if x is not None else 0 for x in in_links]
# outlink = [len(x) if x is not None else 0 for x in out_links]

# sum_in = []
# for x in in_links: 
#     if x is not None:
#         sum_in.append(sum([y[1] for y in x]))
#     else:
#         sum_in.append(0)
# sum_out = []
# for x in out_links:
#     if x is not None:
#         sum_out.append(sum([y[1] for y in x]))
#     else:
#         sum_out.append(0)

# rel = [True if x in rel_docs else False for x in linker.ids()]

# labels = [x for x in range(1, buckets+1)]

# len_df = pd.DataFrame({'id': [x.upper() for x in doc_lens.keys()], 'lens': list(doc_lens.values())}).set_index(['id'])

In [9]:
inter = Interpolater(os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'), normalize=True)

dfs = []
for d in display_names[1:]: 
    interped_dfs = []
    for _lambda in np.arange(0, 1.0, 0.01):
        inter.interpolate(os.path.join(BASE_DIR, 'case-topics-{0}.run'.format(d)), _lambda, 'tmp.run')
        interped_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'])[0])
    dfs.append(interped_dfs)

In [10]:
to = 15
link_text_fig = plot_tune_1d_comp(['base', 'sections', 'legislation'], metrics, 
                    [[base_df for x in range(to+1)]] + [x[:to+1] for x in dfs[:2]], 0.00, (to)/100, 0.01, legend_x=0.965, ylims=ylims, styles=['--'])
link_text_fig.savefig('figures/ausnl-legref-interp.pdf')

<Figure size 1152x432 with 6 Axes>

In [11]:
to = 99
link_text_fig = plot_tune_1d_comp(['base', 'date'], metrics, 
                    [[base_df for x in range(to+1)]] + [x[:to+1] for x in dfs[2:3]], 0.00, (to)/100, 0.01, legend_x=0.965, ylims=ylims, styles=['--'])
link_text_fig.savefig('figures/ausnl-date-interp.pdf')

<Figure size 1152x432 with 6 Axes>

In [12]:
to = 40
link_text_fig = plot_tune_1d_comp(['base', 'hierarchy'], metrics, 
                    [[base_df for x in range(to+1)]] + [x[:to+1] for x in dfs[3:4]], 0.00, (to)/100, 0.01, legend_x=0.965, ylims=ylims, styles=['--'])
link_text_fig.savefig('figures/ausnl-hierarchy-interp.pdf')

<Figure size 1152x432 with 6 Axes>

In [13]:
inter = Interpolater(os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'), normalize=True)

q_dfs = []
for d in display_names[1:]: 
    interped_dfs = []
    for _lambda in np.arange(0, 1.0, 0.01):
        inter.interpolate(os.path.join(BASE_DIR, 'case-topics-{0}.run'.format(d)), _lambda, 'tmp.run')
        interped_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'], per_query=True)[0])
    q_dfs.append(interped_dfs)

In [14]:
ntlm_df = pd.DataFrame(columns=metrics)

for ab, runs in zip(['sec', 'leg', 'date', 'hierarchy'], q_dfs):
    cross = cross_validation(runs, tt_folds, metrics, base_qry)
    ntlm_df.loc[ab] = cross[0]

In [15]:
ntlm_df.loc['$R$'] = base_df.round(4)

date_df = ntlm_df.reindex(['$R$', 'date'])
write_table('tables/ausnl-date', bold_max(date_df).rename(columns=metrics).drop('Unjudged@20',axis='columns').to_latex(escape=False))

In [16]:
leg_df = ntlm_df.reindex(['$R$', 'sec', 'leg'])
write_table('tables/ausnl-leg', bold_max(leg_df).rename(columns=metrics).drop('Unjudged@20',axis='columns').to_latex(escape=False))

In [17]:
h_df = ntlm_df.reindex(['$R$', 'hierarchy'])
write_table('tables/ausnl-hierarchy', bold_max(h_df).rename(columns=metrics).drop('Unjudged@20',axis='columns').to_latex(escape=False))

In [18]:
display_names

['base', 'sec-count', 'leg-count', 'date', 'hierarchy']

In [19]:
link_max = select_1d_max_with_interp(display_names[1:], dfs, 0.0, 0.01, '$\lambda$', inter, base_qry, base_df, 1050, os.path.join(BASE_DIR, 'case-topics-{0}.run'), config.AUS_QREL_PATH, config.AUS_REL_LEVEL, metrics=metrics).T

In [20]:
# print(link_max.drop(['Unjudged@20'], axis='columns').to_latex(escape=False))
link_max

RR   ERR@20     R@20     NDCG      RBP Unjudged@20
base      $\lambda$  1050.00  1050.00  1050.00  1050.00  1050.00     1050.00
          -           0.5686   0.3794   0.4600   0.4613   0.3113      0.0000
date      $\lambda$     0.39     0.39     0.86     0.72     0.72        0.99
          -           0.5702   0.3800   0.4724   0.4622   0.3117     5.46316
hierarchy $\lambda$     0.21     0.23     0.12     0.25     0.00        0.98
          -           0.5739   0.3883   0.4798   0.4641   0.3113         6.6
leg-count $\lambda$     0.00     0.00     0.02     0.00     0.00        0.93
          -           0.5686   0.3794   0.4613   0.4613   0.3113     8.38947
sec-count $\lambda$     0.05     0.06     0.02     0.05     0.00        0.93
          -           0.5940   0.3889   0.4642   0.4740   0.3113     7.77895

In [21]:
# max for err@20
om = copy.copy(config.METRIC_NAMES)
del om['recall_100']
del om['unjudged@20']

for r in display_names[1:]:
    interps = link_max.loc[r, '$\lambda$'].values
    cols = om.keys()
    a = pd.DataFrame()
    for i, c in zip(interps, cols):
        inter.interpolate(os.path.join(BASE_DIR, 'case-topics-{0}.run'.format(r)), float(i), 'tmp.run')
        a[c] = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'], per_query=True)[0][c]

    qry_comp_df = a-base_qry
    qry_comp_fig = qry_comp_df[om.keys()].rename(metrics, axis='columns').plot.box(fontsize=15, boxprops=dict(linestyle='-', linewidth=2), medianprops=dict(linestyle='-', linewidth=2), color=dict(boxes='black', whiskers='black', medians='b', caps='r'), figsize=(16, 4)).axhline(y=0, xmin=0.0, xmax=1.0, linestyle='--', linewidth=1.0, color='grey')
    qry_comp_fig.get_figure().savefig('figures/ausnl-{0}-qry-comp.pdf'.format(r))

/home/danlocke/.local/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/home/danlocke/.local/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/home/danlocke/.local/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with d

<Figure size 1152x288 with 1 Axes>

<Figure size 1152x288 with 1 Axes>

<Figure size 1152x288 with 1 Axes>

<Figure size 1152x288 with 1 Axes>

## Combining multiple features

In [45]:
max_runs = [os.path.join('/home/danlocke/go/src/gridsearch', '{0}-combine-max.run'.format(x)) for x in ['RR', 'ERR', 'R@20', 'NDCG', 'RBP']]
print(max_runs)
max_run_res = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', max_runs)

['/home/danlocke/go/src/gridsearch/RR-combine-max.run', '/home/danlocke/go/src/gridsearch/ERR-combine-max.run', '/home/danlocke/go/src/gridsearch/R@20-combine-max.run', '/home/danlocke/go/src/gridsearch/NDCG-combine-max.run', '/home/danlocke/go/src/gridsearch/RBP-combine-max.run']


CalledProcessError: Command '['trec_eval', '-q', '-m', 'recall.20,100', '-m', 'ndcg', '-m', 'recip_rank', '-m', 'set_P', '-m', 'set_recall', '-m', 'relstring.20', '-m', 'set_F..5', '-l', '1', '/home/danlocke/go/src/github.com/dan-locke/phd-data/aus/filtered-qrels.txt', '/home/danlocke/go/src/gridsearch/RR-combine-max.run']' returned non-zero exit status 2.

In [ ]:
pd.DataFrame(max_run_res, columns=om)

In [ ]:

max_runs = [os.path.join('/home/danlocke/go/src/gridsearch', 'dpbertoprobhierfact-{0}-combine-max.run'.format(x)) for x in ['RR', 'ERR', 'R@20', 'NDCG', 'RBP']]
print(max_runs)
max_run_res = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', max_runs)

In [ ]:
pd.DataFrame(max_run_res, columns=om)

In [ ]:
# def create_folds(queries, k: int):
#     l = len(queries)
#     q_set = set(queries)
#     inds = [x for x in range(l)]
#     np.random.shuffle(inds)
#     queries = np.array(queries)
#     queries = queries[inds]
    
#     avg = l / float(k)
#     parts = []
#     last = 0.0
    
#     test = []

#     while last < l:
#         test.append(list(queries[int(last):int(last + avg)]))
#         last += avg
     
#     for i in range(len(parts)):
#         train.append()
    
#     train = []
#     for i in range(len(test)):
#         train.append(list(q_set.difference(test[i])))
    
#     return train, test
    
# train, test = create_folds(list(queries.keys()), 5)

In [ ]:
# # write folds to file 
# with open('ausnl-folds.txt', 'w') as f:
#     for tr, te in zip(train, test):
#         f.write('{0} {1}\n'.format(tr, te))

In [46]:
tt_folds = read_folds('ausnl-folds.txt')

In [47]:
grid_dir = os.path.join(os.environ["HOME"], "go/src/gridsearch")

grid_name = 'bert-{0}-fold-{1}-combine-max.run'
cros_val_name = 'bert'
#grid_name = 'dpbtsoph-{0}-fold-{1}-combine-max.run'

runs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', [os.path.join(grid_dir, grid_name.format(y, x)) for x in range(0, 5) for y in ['RR', 'ERR', 'R@20', 'NDCG', 'RBP']], per_query=True)

CalledProcessError: Command '['trec_eval', '-q', '-m', 'recall.20,100', '-m', 'ndcg', '-m', 'recip_rank', '-m', 'set_P', '-m', 'set_recall', '-m', 'relstring.20', '-m', 'set_F..5', '-l', '1', '/home/danlocke/go/src/github.com/dan-locke/phd-data/aus/filtered-qrels.txt', '/home/danlocke/go/src/gridsearch/bert-RR-fold-0-combine-max.run']' returned non-zero exit status 2.

In [48]:
names = [os.path.join(grid_dir, grid_name.format(y, x)) for x in range(0, 5) for y in ['RR', 'ERR', 'R@20', 'NDCG', 'RBP']]

In [49]:
# [names[x:x+5] for x in range(0, len(names), 5)]

## results for test dataset for runs

In [50]:
test_res = []
for r in range(5):
    test_res.append([])
run_folds = [runs[x:x+5] for x in range(0, len(runs), 5)]
for filters, f in zip(tt_folds, run_folds):
    for i, measure in enumerate(f):
        test_res[i].append(measure[measure.index.isin(filters[1])].mean())

NameError: name 'runs' is not defined

In [51]:
metrics = copy.copy(config.METRIC_NAMES)
del metrics['unjudged@20']
del metrics['recall_100']

test_res_avg = pd.DataFrame()
for m, r in zip(metrics, test_res):
    test_res_avg[m] = pd.DataFrame(r)[m]
    print('-'*20)
    
test_res_avg = test_res_avg.rename(metrics, axis='columns')

KeyError: 'recip_rank'

In [29]:
fig, axs = plt.subplots(1, 5)
fig.set_size_inches(16, 10)
for i, m in enumerate(metrics.items()):
    print(m)
    p = test_res_avg[m[1]].plot.box(ax=axs[i], fontsize=15, boxprops=dict(linestyle='-', linewidth=2), medianprops=dict(linestyle='-', linewidth=2), color=dict(boxes='black', whiskers='black', medians='b', caps='r'), figsize=(16, 4))
    p.axhline(y=base_df[m[0]], xmin=0.0, xmax=1.0, linestyle='--', linewidth=1.0, color='blue')
    p.axhline(y=sdm_base_df[m[0]], xmin=0.0, xmax=1.0, linestyle='--', linewidth=1.0, color='green')
    axs[i].set_ylim(top=ylims[i])

fig.tight_layout()
fig.savefig('figures/ausnl-{0}-cross-val.pdf'.format(cros_val_name))

('recip_rank', 'RR')
('err@20', 'ERR@20')
('recall_20', 'R@20')
('ndcg', 'NDCG')
('rbp@0.80', 'RBP')


<Figure size 1152x288 with 5 Axes>

In [30]:
# import numpy as np
# train, test = create_folds([1, 3, 9, 19, 23, 25, 31, 37, 39, 45, 51, 97], 5)

In [31]:
# train

In [32]:
# test

In [33]:
# # write folds to file 
# with open('sigir-folds.txt', 'w') as f:
#     for tr, te in zip(train, test):
#         f.write('{0} {1}\n'.format(tr, te))

In [52]:
dfs = []
model_names = ['BERT', '4ft', '5ft']
metric_names = ['RR', 'ERR', 'R@20', 'NDCG', 'RBP']
metric_ll = ['recip_rank', 'err@20', 'recall_20', 'ndcg', 'rbp@0.80']
for m in metric_names:
    dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', 
                        [os.path.join('/home/danlocke/go/src/gridsearch', '{0}-{1}-combine-max.run'.format(r, m)) for r in ['bert', '4feat', '5feat']], per_query=True))

In [70]:
print(len(dfs[0]))
dfs[0][2].mean()[metrics]

3


recip_rank    0.575992
err@20        0.396423
recall_20     0.493566
ndcg          0.479514
rbp@0.80      0.334835
dtype: float64

In [71]:
out_df = pd.DataFrame(columns=metric_names)
for d, m, l in zip(dfs, metric_names, metric_ll):
    out_df[m] = [x[l].mean() for x in d]
    
    print([stats.ttest_rel(d[0][l], x[l]).pvalue for x in d[1:]])
    
out_df.index = model_names
print(out_df.round(4).to_latex())

write_table('tables/ausnl-multi', bold_max(out_df.round(4)).rename(columns=metrics).to_latex(escape=False))

[0.3200807232880815, 0.09836810782778167]
[nan, nan]
[0.8262201175488094, 0.6643414148237861]
[0.8822134444602492, 0.28804322449634245]
[0.7726596470891762, 0.9561487858720811]
\begin{tabular}{lrrrrr}
\toprule
{} &      RR &     ERR &    R@20 &    NDCG &     RBP \\
\midrule
BERT &  0.6239 &  0.4004 &  0.5006 &  0.4850 &  0.3513 \\
4ft  &  0.6064 &  0.3891 &  0.4988 &  0.4841 &  0.3526 \\
5ft  &  0.5760 &  0.3899 &  0.4970 &  0.4783 &  0.3511 \\
\bottomrule
\end{tabular}



In [37]:
tmp_df = pd.DataFrame(columns=metrics)
for n, d in zip(model_names, dfs):
    cross = cross_validation(d, tt_folds, metrics, base_qry)
    tmp_df.loc[n] = cross[0]
    
write_table('tables/ausnl-multi', bold_max(tmp_df).rename(columns=metrics).to_latex(escape=False))

In [17]:
dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', [os.path.join('/home/danlocke/go/src/gridsearch', '{0}-RBP-combine-max.run'.format(r)) for r in ['bert', '4feat', '5feat']], per_query=True)

In [6]:
def read_grid_res(path: str, folds: int):
    with open(path) as f:
        measures = f.readline().split()
        fold_res = [[] for x in measures]
        weights = [[] for x in measures]
        # read the fold score
        for i in range(folds):
            parts = f.readline().split()
            for j in range(len(parts)):
                fold_res[j].append(float(parts[j]))
            
        #read the weights
        for i in range(folds):
            parts = f.readline().split('] [')
            parts = [x.strip(' []\n') for x in parts]
            for j in range(len(parts)):
                weights[j].append([float(x) for x in parts[j].split()])
                
        return measures, fold_res, weights
             
_4res = read_grid_res(os.path.join('/home/danlocke/go/src/gridsearch', '4feat-results.txt'), 5)

In [7]:
_5res = read_grid_res(os.path.join('/home/danlocke/go/src/gridsearch', '5feat-results.txt'), 5)

In [8]:
bertres = read_grid_res(os.path.join('/home/danlocke/go/src/gridsearch', 'bert-results.txt'), 5)

In [9]:
rev_metrics = {v:k for k,v in metrics.items()}
rev_metrics['ERR'] = rev_metrics['ERR@20']

In [13]:
rev_metrics

{'RR': 'recip_rank',
 'ERR@20': 'err@20',
 'R@20': 'recall_20',
 'NDCG': 'ndcg',
 'RBP': 'rbp@0.80',
 'Unjudged@20': 'unjudged@20',
 'ERR': 'err@20'}

In [17]:
fig, axs = plt.subplots(1, 5)
fig.set_size_inches(16, 10)
cnt = 0
for i in [2, 3, 4, 1, 0]:
    a = pd.DataFrame()
    a['x'] = [0 for x in range(5)]
    a['x1'] = [1 for x in range(5)]
    a['x2'] = [2 for x in range(5)]
    a['bert'] = bertres[1][i]
    a['4ft'] = _4res[1][i]
    a['5ft'] = _5res[1][i]
    p = a.plot.scatter(y='bert', x='x', ax=axs[cnt], fontsize=15, figsize=(16, 4))
    a.plot.scatter(y='4ft', x='x1', ax=axs[cnt], fontsize=15, figsize=(16, 4))
    a.plot.scatter(y='5ft', x='x2', ax=axs[cnt], fontsize=15, figsize=(16, 4))
    p.axhline(y=base_df[rev_metrics[_4res[0][i]]], xmin=0.0, xmax=1.0, linestyle='--', linewidth=1.0, color='b')
    p.set(xlabel=None)
    p.set(xticks=[0, 1, 2])
    p.set_ylabel(_4res[0][i], fontsize=15)
    p.set_xticklabels(['bert', '4ft', '5ft'], rotation='vertical', fontsize=18)
    cnt += 1

fig.tight_layout()
fig.savefig('figures/ausnl-comb-train-res.pdf')

*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB o

<Figure size 1152x288 with 5 Axes>

In [131]:
fig, axs = plt.subplots(1, 5)
fig.set_size_inches(16, 10)
for i, m in enumerate(_4res[0]):
    a = pd.DataFrame()
#     a['x'] = [0 for x in range(5)]
# #     a['x1'] = [1 for x in range(5)]
# #     a['x2'] = [2 for x in range(5)]
# #     a['bert'] = bertres[2][i]
# #     a['4ft'] = _4res[2][i]
#     a['5ft'] = _5res[2][i]
#     print(a)
#     break
#     p = a.plot.scatter(y='bert', x='x', ax=axs[i], fontsize=15, figsize=(16, 4))
#     a.plot.scatter(y='4ft', x='x1', ax=axs[i], fontsize=15, figsize=(16, 4))
#     a.plot.scatter(y='5ft', x='x2', ax=axs[i], fontsize=15, figsize=(16, 4))
#     p.axhline(y=base_df[rev_metrics[m]], xmin=0.0, xmax=1.0, linestyle='--', linewidth=1.0, color='b')
#     p.set(xlabel=None)
#     p.set(xticks=[0, 1, 2])
#     p.set_ylabel(m, fontsize=15)
#     p.set_xticklabels(['bert', '4ft', '5ft'], rotation='vertical', fontsize=18)

   x                             5ft
0  0  [0.29, 0.51, 0.12, 0.01, 0.07]
1  0  [0.28, 0.54, 0.08, 0.04, 0.06]
2  0   [0.3, 0.53, 0.09, 0.01, 0.07]
3  0    [0.33, 0.5, 0.1, 0.05, 0.02]
4  0  [0.28, 0.54, 0.08, 0.04, 0.06]


<Figure size 1152x720 with 5 Axes>

In [18]:
all_weight_df = pd.DataFrame()

for i in range(5):
    tmp = pd.DataFrame(_4res[2][i])
    tmp['measure'] = _4res[0][i]
    tmp['x'] = 0
    all_weight_df = all_weight_df.append(tmp)
    
# all_weight_df.plot.scatter(x='measure', y=0)

In [31]:
tmp = pd.DataFrame()

In [41]:
tmp['0cnt'] = all_weight_df.groupby('measure')[0].value_counts()
tmp['1cnt'] = all_weight_df.groupby('measure')[1].value_counts()

In [42]:
tmp

0cnt  1cnt
measure 0               
ERR     0.26     2   NaN
        0.30     1   NaN
        0.46     1   NaN
        0.55     1   NaN
NDCG    0.46     2   NaN
        0.52     2   NaN
        0.32     1   NaN
R@20    0.52     3   NaN
        0.46     1   NaN
        0.49     1   NaN
RBP     0.29     2   NaN
        0.32     2   NaN
        0.34     1   NaN
RR      0.28     2   NaN
        0.13     1   NaN
        0.25     1   NaN
        0.26     1   NaN

In [40]:

def create(res, feature_names, folds=5):
    xs = [[x]*5 for x in range(len(feature_names))]
    
    df = pd.DataFrame()

    for i in range(5):
        tmp = pd.DataFrame(res[2][i])
        tmp['measure'] = res[0][i]
        tmp['x'] = 0
        df = df.append(tmp)

    df2 =pd.DataFrame()
    for i in range(len(feature_names)):
        df2[i] = df.groupby('measure')[i].value_counts()
    
    fig, axs = plt.subplots(1, 5)
    fig.set_size_inches(16, 6)
    inds = [2, 3, 4, 1, 0]
    for i, ind in enumerate(inds):
        print(df2)
#         print(ind, res[0][ind])
        tmp = df2[df2['measure'] == res[0][ind]]
        break

#         tmp = df[df['measure'] == res[0][ind]]
# #         print(tmp)
#         for j in range(len(feature_names)):
#             legend = False
#             sns.scatterplot(xs[0], tmp[j], alpha=0.8, ax=axs[i], markers=['X'], style=0, legend=legend, s=40)
#         axs[i].set_xticks([0])
#         axs[i].set_ylabel(res[0][ind], size=20)
#         axs[i].set_ylim(0, 1)
#         axs[i].set_xticks([])
#         fig.legend(feature_names, frameon=True, ncol=1, bbox_to_anchor=[0.99, 0.97], prop={"size": 12}, markerfirst=False).get_frame().set_edgecolor('black')
    
    fig.tight_layout()
    
    return fig
    
fig = create(_4res, ['DP', 'BERT', 'court', 'oprob'])
# fig.savefig('figures/ausnl-4ft-weights.pdf')


# ticks = [str(int(x)) for x in bucket_df.index.categories.left.tolist()]
# ax.set_xticks(range(len(ticks)))
# ax.set_xticklabels(ticks, rotation=270)

# ax.tick_params(labelsize=15)

              0   1    2   3
measure 0                   
ERR     0.26  2 NaN  1.0 NaN
        0.30  1 NaN  NaN NaN
        0.46  1 NaN  NaN NaN
        0.55  1 NaN  NaN NaN
NDCG    0.46  2 NaN  NaN NaN
        0.52  2 NaN  NaN NaN
        0.32  1 NaN  NaN NaN
R@20    0.52  3 NaN  NaN NaN
        0.46  1 NaN  NaN NaN
        0.49  1 NaN  NaN NaN
RBP     0.29  2 NaN  NaN NaN
        0.32  2 NaN  NaN NaN
        0.34  1 NaN  NaN NaN
RR      0.28  2 NaN  NaN NaN
        0.13  1 NaN  NaN NaN
        0.25  1 NaN  NaN NaN
        0.26  1 NaN  NaN NaN


KeyError: 'measure'

<Figure size 1152x432 with 5 Axes>

In [16]:
fig = create(_5res, ['DP', 'BERT', 'court', 'oprob', 'fact'])
fig.savefig('figures/ausnl-5ft-weights.pdf')

2 RR
3 ERR
4 R@20
1 NDCG
0 RBP


/home/danlocke/.local/lib/python3.8/site-packages/seaborn/_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(
/home/danlocke/.local/lib/python3.8/site-packages/seaborn/_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(
/home/danlocke/.local/lib/python3.8/site-packages/seaborn/_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(
/

<Figure size 1152x432 with 5 Axes>

In [17]:
fig = create(bertres, ['DP', 'BERT'])
fig.savefig('figures/ausnl-bert-weights.pdf')

2 RR
3 ERR
4 R@20
1 NDCG
0 RBP


/home/danlocke/.local/lib/python3.8/site-packages/seaborn/_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(
/home/danlocke/.local/lib/python3.8/site-packages/seaborn/_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(
/home/danlocke/.local/lib/python3.8/site-packages/seaborn/_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(
/

<Figure size 1152x432 with 5 Axes>